In [1]:
# IMPORTING NECESSARY LIBRARIES
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import plotly.express as px
import plotly.graph_objects as go
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import time
import re


C:\Users\Dell\AppData\Local\Temp\ipykernel_19148\3923802737.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import requests                   
from bs4 import BeautifulSoup     
import pandas as pd   

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
# SCRAPING DATA
base_url = "https://www.amazon.in/Allen-Solly-Regular-AMKP317G04249_Jet-Black_Large/product-reviews/B06Y2FG6R7/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
pages = 50
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):
    
    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"data-hook": "review"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")



Scraping page 1
   ---> 0 total reviews
Scraping page 2
   ---> 10 total reviews
Scraping page 3
   ---> 20 total reviews
Scraping page 4
   ---> 20 total reviews
Scraping page 5
   ---> 20 total reviews
Scraping page 6
   ---> 20 total reviews
Scraping page 7
   ---> 20 total reviews
Scraping page 8
   ---> 20 total reviews
Scraping page 9
   ---> 20 total reviews
Scraping page 10
   ---> 20 total reviews
Scraping page 11
   ---> 20 total reviews
Scraping page 12
   ---> 20 total reviews
Scraping page 13
   ---> 20 total reviews
Scraping page 14
   ---> 20 total reviews
Scraping page 15
   ---> 20 total reviews
Scraping page 16
   ---> 20 total reviews
Scraping page 17
   ---> 20 total reviews
Scraping page 18
   ---> 20 total reviews
Scraping page 19
   ---> 20 total reviews
Scraping page 20
   ---> 20 total reviews
Scraping page 21
   ---> 20 total reviews
Scraping page 22
   ---> 20 total reviews
Scraping page 23
   ---> 30 total reviews
Scraping page 24
   ---> 40 total reviews
Sc

In [5]:
# PUTTING DATA INTO DATAFRAME
df = pd.DataFrame()
df["reviews"] = reviews
df.head()

,reviews
0,sujeeth5.0 out of 5 stars\nPerfect fit\nReview...
1,Placeholder4.0 out of 5 stars\ngood product\nR...
2,sonia5.0 out of 5 stars\ngood product with bad...
3,N P.3.0 out of 5 stars\nSize L is too big for ...
4,"Himanshu garg5.0 out of 5 stars\nBest fabric, ..."


In [6]:
# FUNCTION FOR CLEANING DATA
def clean(text):
    text = re.sub('[^A-Za-z]+', ' ', str(text))
    text = re.sub('^(.*stars )'," ", str(text))
    return text



In [7]:
# PUTTING CLEANED DTA INTO DATAFRAME
df1 = pd.DataFrame()
df1["cleaned reviews"] = df["reviews"].apply(clean)
df1.head()

,cleaned reviews
0,Perfect fit Reviewed in India on March Size X...
1,good product Reviewed in India on March Size ...
2,good product with bad packaging Reviewed in I...
3,Size L is too big for me Please replace this ...
4,Best fabric color and fitting Reviewed in Ind...


In [8]:
# POS TAGGING THE DATA
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    l = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
          l.append(tuple([word, pos_dict.get(tag[0])]))
    return l 

df1["POS tagged"] = df1["cleaned reviews"].apply(token_stop_pos)
df1.head()

,cleaned reviews,POS tagged
0,Perfect fit Reviewed in India on March Size X...,"[(Perfect, a), (fit, n), (Reviewed, n), (India..."
1,good product Reviewed in India on March Size ...,"[(good, a), (product, n), (Reviewed, n), (Indi..."
2,good product with bad packaging Reviewed in I...,"[(good, a), (product, n), (bad, a), (packaging..."
3,Size L is too big for me Please replace this ...,"[(Size, n), (L, n), (big, a), (Please, n), (re..."
4,Best fabric color and fitting Reviewed in Ind...,"[(Best, n), (fabric, n), (color, n), (fitting,..."


In [9]:
# LEMMATIZING DATA
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
        if not pos:
            lemma = word
            lemma_rew = lemma_rew + " " + lemma
        else:
            lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
            lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

df1['Lemma'] = df1['POS tagged'].apply(lemmatize)
df1.head()

,cleaned reviews,POS tagged,Lemma
0,Perfect fit Reviewed in India on March Size X...,"[(Perfect, a), (fit, n), (Reviewed, n), (India...",Perfect fit Reviewed India March Size XLColo...
1,good product Reviewed in India on March Size ...,"[(good, a), (product, n), (Reviewed, n), (Indi...",good product Reviewed India March Size XLCol...
2,good product with bad packaging Reviewed in I...,"[(good, a), (product, n), (bad, a), (packaging...",good product bad packaging Reviewed India Ma...
3,Size L is too big for me Please replace this ...,"[(Size, n), (L, n), (big, a), (Please, n), (re...",Size L big Please replace item size Reviewed...
4,Best fabric color and fitting Reviewed in Ind...,"[(Best, n), (fabric, n), (color, n), (fitting,...",Best fabric color fitting Reviewed India Mar...


In [10]:
# ANALYZING THE SENTIMENT OF THE REVIEWS AND GIVING IT A SCORE
analyzer = SentimentIntensityAnalyzer()
def vadersentimentanalysis(review):
    vs = analyzer.polarity_scores(review)
    return vs['compound']
df1['Sentiment'] = df1['Lemma'].apply(vadersentimentanalysis)
def vader_analysis(compound):
    if compound >= 0.7:
        return 'Positive'
    elif compound < 0.4 :
        return 'Negative'
    else:
        return 'Neutral'
df1['Analysis'] = df1['Sentiment'].apply(vader_analysis)
df1

,cleaned reviews,POS tagged,Lemma,Sentiment,Analysis
0,Perfect fit Reviewed in India on March Size X...,"[(Perfect, a), (fit, n), (Reviewed, n), (India...",Perfect fit Reviewed India March Size XLColo...,0.9493,Positive
1,good product Reviewed in India on March Size ...,"[(good, a), (product, n), (Reviewed, n), (Indi...",good product Reviewed India March Size XLCol...,0.9300,Positive
2,good product with bad packaging Reviewed in I...,"[(good, a), (product, n), (bad, a), (packaging...",good product bad packaging Reviewed India Ma...,-0.1027,Negative
3,Size L is too big for me Please replace this ...,"[(Size, n), (L, n), (big, a), (Please, n), (re...",Size L big Please replace item size Reviewed...,0.9206,Positive
4,Best fabric color and fitting Reviewed in Ind...,"[(Best, n), (fabric, n), (color, n), (fitting,...",Best fabric color fitting Reviewed India Mar...,0.9468,Positive
5,It s not a black colour guys Reviewed in Indi...,"[(black, a), (colour, n), (guys, n), (Reviewed...",black colour guy Reviewed India February Siz...,0.6486,Neutral
6,Very good fitting and color Reviewed in India...,"[(good, a), (fitting, n), (color, n), (Reviewe...",good fitting color Reviewed India March Size...,0.8481,Positive
7,Expensive but good quality Reviewed in India ...,"[(Expensive, a), (good, a), (quality, n), (Rev...",Expensive good quality Reviewed India March ...,0.6486,Neutral
8,Packaging Reviewed in India on March Size LCo...,"[(Packaging, v), (Reviewed, v), (India, n), (M...",Packaging Reviewed India March Size LColour ...,-0.2732,Negative
9,Best polo t shirt Reviewed in India on March ...,"[(Best, n), (polo, n), (shirt, n), (Reviewed, ...",Best polo shirt Reviewed India March Verifie...,0.8648,Positive


In [21]:
# PLOTTING RESULT IN PIE-CHART
fig=px.pie(df1, df1["Analysis"], title="Review Analysis", color_discrete_sequence=px.colors.qualitative.Pastel2, width=1024, height=768)
fig.update_layout(title_font_size=30)
fig.update_layout(legend=dict(font=dict(size= 30)))
fig.update_traces(textfont_size=30)
fig.show()